In [ ]:
import pandas as pd
import numpy as np
import ast
import cv2
import matplotlib.pyplot as plt

from data_augmenter import duplicate_and_flip_actions, create_action_windows, augment_all_actions, explode_keypoints
from data_visualization import draw_random_window

PATH_KEYPOINTS_FILTERED             = "../Data/tmp/keypoints_filtered.csv"
PATH_METADATA                       = "../Data/Processed/metadata.csv"
PATH_DATA                           = "../Data/Processed/data.csv"

WINDOWS_PER_ACTION                  = 5
JITTER_STD = 2.5
NOISE_STD = 2.5
SCALE_STD = 0.075
ROTATE_DEG = 7

In [ ]:
df_interpolated = pd.read_csv(PATH_KEYPOINTS_FILTERED)
df_interpolated["keypoints"] = df_interpolated["keypoints"].apply(ast.literal_eval)

df_augmented = duplicate_and_flip_actions(
    df_interpolated, 
    image_width=1920
)

df_augmented = create_action_windows(
    df_augmented, 
    window_size=MINIMUM_DURATION, 
    windows_per_action=WINDOWS_PER_ACTION, 
    random_state=42
)

df_augmented = augment_all_actions(df_augmented,
    jitter_std=JITTER_STD,
    noise_std=NOISE_STD,
    scale_std=SCALE_STD,
    rotate_deg=ROTATE_DEG
)

df_augmented = df_augmented[
    ["file", "fencer", "action_id", "window_id", "action", "frame", "start_frame", "end_frame", "keypoints"]
].sort_values(["file", "action_id", "window_id", "frame"]).reset_index(drop=True)

print(df_augmented)
print("")

print("Number of actions before augmentation: ", df_interpolated["action_id"].nunique())
print("Number of actions after augmentation:  ", df_augmented["window_id"].nunique())

In [ ]:
draw_random_window(df_augmented, 3, seed=42)

In [ ]:
df_metadata = df_augmented[["file", "fencer", "action_id", "window_id", "action", "frame", "start_frame", "end_frame"]]
df_metadata.to_csv(PATH_METADATA, index=False)

df_data = df_augmented[["window_id", "frame", "action", "keypoints"]]
df_data.sort_values(["window_id"], inplace=True)

df_data = explode_keypoints(df_data)
df_data.to_csv(PATH_DATA, index=False)